In [13]:
import tensorflow as tf
import pickle
import numpy as np
import cv2


cnn_model = tf.keras.models.load_model('models/kidneystone.keras')


with open("models/svm_model.pkl", "rb") as f:
    svm_model = pickle.load(f)


with open('models/decision_tree_model.pkl', 'rb') as file:
    decision_tree_model = pickle.load(file)


def ensemble_predict(cnn_model, svm_model, decision_tree_model, X):
    def cnn_function():
        yhat = cnn_model.predict(np.expand_dims(X, 0))
        return [np.argmax(yhat[0])] 


    def svm_function():
        svm = svm_model.best_estimator_
        yhat = svm.predict(X.flatten().reshape(1, -1))
        return [yhat[0]] 


    def decision_function():
        decision_tree= decision_tree_model.best_estimator_
        yhat = decision_tree.predict(X.flatten().reshape(1, -1))
        return [yhat[0]]

 
    print("CNN Prediction:", cnn_function())
    print("SVM Prediction:", svm_function())
    print("Decision Tree Prediction:", decision_function())

   
    ensemble_pred = []
    for cnn, svm, dt in zip(cnn_function(), svm_function(), decision_function()):
        votes = [cnn, svm, dt]
        ensemble_pred.append(max(set(votes), key=votes.count))
    return ensemble_pred


class_labels = ['Cyst', 'Normal', 'Stone', 'Tumor']


img = cv2.imread("images/normal.png")  
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
resized_img = cv2.resize(img_rgb, (256, 256))
normalized_img = resized_img / 255.0


ensemble_prediction = ensemble_predict(cnn_model, svm_model, decision_tree_model, normalized_img)
print("Ensemble Prediction:", class_labels[ensemble_prediction[0]])  # Assuming ensemble_pred is a list with a single value




/opt/homebrew/Caskroom/miniconda/base/envs/python3122/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 22 variables whereas the saved optimizer has 42 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 272ms/step
CNN Prediction: [1]
SVM Prediction: [0]
Decision Tree Prediction: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
Ensemble Prediction: Cyst


In [5]:
import tensorflow as tf
import pickle
import numpy as np
import cv2

cnn_model = tf.keras.models.load_model('models/kidneystone.keras')

# Load the SVM model
with open("models/grid_search_model.pkl", "rb") as f:
    svm_model = pickle.load(f)
    # svm_model =svm_model.best_estimator_

# Load the Decision Tree model
with open('models/decision_tree_model.pkl', 'rb') as file:
    decision_tree_model = pickle.load(file)
    # decision_tree_model =decision_tree_model.best_estimator_

def ensemble_predict(cnn_model, svm_model, decision_tree_model, X):
    def cnn_function():
        yhat = cnn_model.predict(np.expand_dims(X, 0))
        return np.argmax(yhat[0])

    def svm_function():
        X_img = X.reshape(1, -1)
        yhat = svm_model.predict(X_img)
        return yhat[0]

    def decision_function():
        X_img = X.reshape(1, -1)
        yhat = decision_tree_model.predict(X_img)
        return yhat[0]

    cnn_pred = cnn_function()
    svm_pred = svm_function()
    dt_pred = decision_function()

    print(cnn_pred, svm_pred, dt_pred)

    # Combine predictions
    ensemble_pred = max(set([cnn_pred, svm_pred, dt_pred]), key=[cnn_pred, svm_pred, dt_pred].count)
    return ensemble_pred

class_labels = ['Cyst', 'Normal', 'Stone', 'Tumor']

# Read and preprocess the new image
img = cv2.imread("images/stone.jpeg")  # Replace "path_to_new_image.jpg" with the path to your new image
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
resized_img = cv2.resize(img_rgb, (256, 256))
normalized_img = resized_img / 255.0

ensemble_prediction = ensemble_predict(cnn_model, svm_model, decision_tree_model, normalized_img)
print("Ensemble Prediction:", class_labels[ensemble_prediction])



/opt/homebrew/Caskroom/miniconda/base/envs/python3122/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 22 variables whereas the saved optimizer has 42 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 315ms/step
2 0 0
Ensemble Prediction: Cyst
